In [1]:
import cv2
import numpy as np

import os

In [2]:
templates_orig = cv2.imread("template.png")
templates = cv2.cvtColor(templates_orig, cv2.COLOR_BGR2GRAY)

In [3]:
def show(img, title="show", wait=0):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 模板提取

In [4]:
# 二值化处理，并对像素去反相
_, temp_threshold = cv2.threshold(templates, 127, 255, cv2.THRESH_BINARY_INV)
# 搜索轮廓
temp_contours, _ = cv2.findContours(temp_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [5]:
show(temp_threshold)

In [6]:
# 求取模板的 bounding-box
temp_bounding_boxes = [cv2.boundingRect(contour) for contour in temp_contours]

In [7]:
# 对坐标进行编码
temp_bounding_boxes.reverse()

In [8]:
temp_bounding_boxes

[(14, 20, 55, 86),
 (93, 20, 55, 86),
 (173, 20, 55, 86),
 (252, 20, 55, 86),
 (338, 20, 49, 86),
 (411, 20, 55, 86),
 (491, 20, 55, 86),
 (571, 20, 54, 86),
 (650, 20, 55, 86),
 (730, 20, 54, 86)]

In [35]:
regions = []

for i in range(len(temp_bounding_boxes)-1):
    boundary = (temp_bounding_boxes[i][0] + temp_bounding_boxes[i+1][0]) / 2
    regions.append(boundary)
    
regions

[53.5, 133.0, 212.5, 295.0, 374.5, 451.0, 531.0, 610.5, 690.0]

In [36]:
def identify(loc):
    num = 9
    for index, boundary in enumerate(regions):
        if loc < boundary:
            num = index
            break
        
    return num

In [10]:
# 查看轮廓的提取效果
tmp = templates_orig.copy()
tmp = cv2.drawContours(tmp, temp_contours, 0, (0, 0, 255), 2)

show(tmp)

In [11]:
# 查看 bounding_box
tmp = templates_orig.copy()

for x, y, w, h in temp_bounding_boxes:
    cv2.rectangle(tmp, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
show(tmp)

## 目标账号识别

In [107]:
path = "images/credit_card_05.png"

card_img = cv2.resize(cv2.imread(path), (580, 370))
# target = cv2.Canny(img, 120, 150)
card_gray = cv2.cvtColor(card_img, cv2.COLOR_BGR2GRAY)
_, card_binary = cv2.threshold(card_gray, 175, 255, cv2.THRESH_BINARY)

# show(card_binary)

In [108]:
card_mask = np.zeros(card_binary.shape, np.uint8)
card_mask[30:340, 30:550] = 255

card_masked = cv2.bitwise_and(card_binary, card_mask)

# show(masked)

In [109]:
# 先腐蚀再膨胀，去较小的字体
conv_kernel = np.ones((9, 9), np.uint8)
opened = cv2.morphologyEx(card_masked, cv2.MORPH_OPEN, conv_kernel)

# 礼帽操作，强化边缘
conv_kernel = np.ones((7, 7), np.uint8)
tophat = cv2.morphologyEx(card_masked-opened, cv2.MORPH_TOPHAT, conv_kernel)

"""# 先腐蚀再膨胀，去较小的字体
conv_kernel = np.ones((3, 3), np.uint8)
opened_ag = cv2.morphologyEx(tophat, cv2.MORPH_OPEN, conv_kernel)

# 腐蚀操作
conv_kernel = np.ones((3, 3), np.uint8)
eroded = cv2.erode(tophat,  conv_kernel, iterations=1)
"""

# 膨胀，将数字块连到一起
conv_kernel = np.ones((1, 21), np.uint8)
dilated = cv2.dilate(tophat, conv_kernel, iterations=3)


# 先膨胀再腐蚀，填充卡号字段的噪点
conv_kernel = np.ones((5, 5), np.uint8)
closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, conv_kernel)

# 对比效果
top = np.hstack((card_masked, opened, card_masked-opened))
bottom = np.hstack((tophat, dilated, closed))
preprocess = np.vstack((top, bottom))
# show(preprocess)

In [110]:
# 进行边缘提取
card_contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [111]:
# 检查边界提取情况
with_contours = card_img.copy()
cv2.drawContours(with_contours, card_contours, -1, (0, 255, 0), 1)
show(with_contours)

In [112]:
card_bounding_boxes = [cv2.boundingRect(contour) for contour in card_contours]

w_downer_limit = 440
h_downer_limit = 18
h_upper_limit = 40

card_number_boxes = []
for x, y, w, h in card_bounding_boxes:
    if w < w_downer_limit or h < h_downer_limit or h > h_upper_limit:
        continue
    
    card_number_boxes.append((x, y, w, h))

In [113]:
# 若存在多个或者不存在 RoI ，则说明提取失败
assert len(card_number_boxes) == 1

In [114]:
# 检查账号提取情况
with_bounding_boxes = card_img.copy()
for x, y, w, h in card_number_boxes:
    cv2.rectangle(with_bounding_boxes, (x, y), (x+w, y+h), (0, 255, 0), 1)

show(with_bounding_boxes)

## 提取卡号部分

In [115]:
# 提取卡号序列的 ROI
seq_x, seq_y, seq_width, seq_height = card_number_boxes[0]

id_img = card_img[seq_y:seq_y+seq_height, seq_x:seq_x+seq_width]
id_gray = cv2.cvtColor(id_img, cv2.COLOR_BGR2GRAY)
_, id_binary = cv2.threshold(id_gray, 160, 255, cv2.THRESH_BINARY)

In [116]:
id_padding = 10
padded_x = seq_x - id_padding
padded_y = seq_y - id_padding
padded_width = seq_width + 2 * id_padding
padded_height = seq_height + 2 * id_padding

id_mask = np.zeros((padded_height, padded_width), dtype=np.uint8)
id_mask[id_padding:-id_padding, id_padding:-id_padding] = id_binary

In [117]:
# 优化数字显示
conv_kernel = np.ones((2, 2), np.uint8)
dilated_id = cv2.dilate(id_mask, conv_kernel, iterations=1)
# dilated_id = cv2.morphologyEx(id_mask, cv2.MORPH_OPEN, conv_kernel)
# dilated_id = id_mask
# show(dilated_id)

In [118]:
# 从账号区提取数字
card_id_contours, _ = cv2.findContours(dilated_id, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
id_bounding_boxes  = [cv2.boundingRect(contour) for contour in card_id_contours]

In [119]:
# 检查数字轮廓提取情况
with_contours = card_img.copy()[padded_y:padded_y+padded_height, padded_x:padded_x+padded_width]
with_contours = cv2.drawContours(with_contours, card_id_contours,-1, (0, 255, 0), 1)

show(with_contours)

In [120]:
# 检查数字bounding-box提取情况
with_bounding_boxes = card_img.copy()[padded_y:padded_y+padded_height, padded_x:padded_x+padded_width]

for x, y, w, h in id_bounding_boxes:
    cv2.rectangle(with_bounding_boxes, (x, y), (x+w, y+h), (0, 255, 0), 1)

show(with_bounding_boxes)

In [121]:
# 模板匹配
number_padding = 5

id_seq = []
for x, y, w, h in id_bounding_boxes:
#     x -= number_padding
#     y -= number_padding
#     w += 2 * number_padding
#     h += 2 * number_padding
    num = dilated_id[y:y+h, x:x+w]
    num = cv2.resize(num, (55, 86))
    
    # 模板匹配
    matches = cv2.matchTemplate(temp_threshold, num, cv2.TM_CCOEFF_NORMED)

    # 搜索最佳匹配项
    min_val, max_val, min_loc, (max_loc, _) = cv2.minMaxLoc(matches)

#     show(num)
    res = identify(max_loc)
    
    id_seq.append(str(res))
    
id_seq.reverse()
print("卡号 :", "".join(id_seq))
show(card_img)

卡号 : 5476767898765432
